# Randomized Response Algorithms

The purpose of this supporting material is to show how one can still do meaningful data analysis on noisy data.

In a binary example and a normal gaussian example, we draw true data points from a distrubution. We add noise to the true dataset in creating a collected dataset. Finally we use the collected dataset to draw inferences about the original distributions.

Here we demonstrate that with enough data sample, the noise -- and consequently the privacy -- added to each data point might stop mattering with large N.

In [24]:
import numpy as np


In [25]:
# Returns true or false, each with .5 probability
def flip_coin():
    return np.random.binomial(1, .5) == 1

print(flip_coin())
print(flip_coin())
print(flip_coin())


True
False
True


In [26]:
p = .42 # this is the parameter we're trying to predict
N = 100

# Draw N samples from a bernoilli distribution parameterized by p
true_data = np.random.binomial(1, p, size = (N))

# Here's how we collect data
# First flip a coin. If it lands heads, respond truthfully
# Otherwise, flip a second coin. If it lands heads, respond True. If it lands tails, respond False.
collected_data = []
for x in true_data:
    if flip_coin():
        collected_data.append(x)
    else:
        if flip_coin():
            collected_data.append(1)
        else:
            collected_data.append(0)

In [27]:
# Since the P(response = 1) = (1/2)p + 1/4
# the expected value of a response is = (1/2)p + 1/4.
# We can equate this to the mean of the responses and solve for p.

estimated_p = (np.array(collected_data).mean() - .25) / (.5)
print(estimated_p)

0.3


In [29]:
# Let's explore how sample size affects our result.

for N in [1e2, 1e3, 1e4, 1e5, 1e6, 1e7]:
    true_data = np.random.binomial(1, p, size = (int(N)))
    collected_data = []
    for x in true_data:
        if flip_coin():
            collected_data.append(x)
        else:
            if flip_coin():
                collected_data.append(1)
            else:
                collected_data.append(0)
    estimated_p = (np.array(collected_data).mean() - .25) / (.5)
    print("N=", N, "p =", estimated_p)



N= 100.0 p = 0.3
N= 1000.0 p = 0.382
N= 10000.0 p = 0.401
N= 100000.0 p = 0.41932
N= 1000000.0 p = 0.419852
N= 10000000.0 p = 0.420168


In [30]:
# Let's simulate a two level randomized response scheme
# Here's let's draw samples from a normal distribution. These will be the true values.
# Then we will add gaussian noise to each sample.
# Finally we'll add a second layer of gaussian noise to each sample.
# Can we use this final layer to infer the mean of the original normal distribution?

mu = 42
sigma_0 = 10
sigma_1 = 5
sigma_2 = 5
N = 1000
true_data = np.random.normal(mu, sigma_0, size = (N))

first_randomization = []
for x in true_data:
    first_randomization.append(np.random.normal(x, sigma_1))
    
collected_data = []
for x in first_randomization:
    collected_data.append(np.random.normal(x, sigma_2))

predicted_mu = np.array(collected_data).mean()
print("predicted mu:", predicted_mu, "actual mu:", mu)

predicted mu: 42.2296687733 actual mu: 42


In [32]:
# Similarly, let's observe how sample size affects this

mu = 42
sigma_0 = 10
sigma_1 = 5
sigma_2 = 5

for N in [1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7]:
    true_data = np.random.normal(mu, sigma_0, size = (int(N)))

    first_randomization = []
    for x in true_data:
        first_randomization.append(np.random.normal(x, sigma_1))

    collected_data = []
    for x in first_randomization:
        collected_data.append(np.random.normal(x, sigma_2))

    predicted_mu = np.array(collected_data).mean()
    print("N", N, "predicted mu:", predicted_mu)


N 1.0 predicted mu: 61.1815681318
N 10.0 predicted mu: 38.4513594696
N 100.0 predicted mu: 40.2892790992
N 1000.0 predicted mu: 41.8736242943
N 10000.0 predicted mu: 41.8844012519
N 100000.0 predicted mu: 41.9549832944
N 1000000.0 predicted mu: 42.0074336456
N 10000000.0 predicted mu: 42.0073227133
